<a href="https://colab.research.google.com/github/gecrooks/quantumflow-dev/blob/colab/notebooks/qf-tutorial-translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QuantumFlow Tutorial: Translating quantum gates

In [12]:
!pip install -qqq git+https://github.com/gecrooks/quantumflow.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


[QuantumFlow](https://https://github.com/gecrooks/quantumflow-dev) contains a rich collection of quantum gates. But often we want to express quantuym circuits using a restricted set of gates. We'll use the controlled-swap gate as an example.

In [13]:
import quantumflow as qf
circ_cswap = qf.Circuit(qf.CSwap(0, 1, 2))
circ_cswap

Lets convert this CSwap into a circuit containing CZ, T, and H gates. We chain three seperate translations: a conversion of a controlled swap to a controlled-controlled not (plus controlled nots); a conversion of CCNots to CNots (plus 1-qubit gates); and a conversion of controlled-nots to controlled-Z's. These translations are applied in turn.

In [14]:

translators = [
    qf.translate_cswap_to_ccnot,
    qf.translate_ccnot_to_cnot,
    qf.translate_cnot_to_cz,
]
circ_translated0 = qf.circuit_translate(circ_cswap, translators)
circ_translated0


Lets check that these two circuits are in fact equivelant. The function `circuits_close` applies each circuit to random initial quantum states and checks that we get the same outout state in each case. 

In [15]:
qf.circuits_close(circ_cswap, circ_translated0)

True

Alternatively, we can convert each circuit to a 3-qubit gate, and check that the gate operators are equivalent. In both cases we're insensitive to physically meaningless differences in phase.


In [16]:
qf.gates_close(circ_cswap.asgate(), circ_translated0.asgate())

True

There are over 100 translations in `QuantumFlow`. Fortunatly you don't generally need to access them explicetly.

In [17]:
len(qf.TRANSLATIONS)

128

Instead of specifing the translations, we can instead specify the set of gates to target the translate at.

In [18]:
circ_translated1 = qf.circuit_translate(circ_cswap, targets=[qf.CZ, qf.H, qf.T, qf.T_H])
assert qf.circuits_close(circ_cswap, circ_translated1)
circ_translated1

Here `qf.T_H` is the Hermitian conjugate of the T-gate. 

If we don't specify the targets, then a default gate set is targeted (given by `qf.TERMINAL_GATES`).

In [19]:
circ_translated2 = qf.circuit_translate(circ_cswap)
assert qf.circuits_close(circ_cswap, circ_translated2)
circ_translated2

Different quantum libraries support different sets of quantum gates. For instance `qiskit` does not support the fermionic simulator gate. But we can translate the fsim gate into a circuit with gates that qiskit does understand.

In [20]:
circ_fsim = qf.Circuit(qf.FSim(0.1, 0.2,  0, 1))
circ_fsim_qiskit = qf.circuit_translate(circ_fsim, targets=qf.QISKIT_GATES)
assert qf.circuits_close(circ_fsim, circ_fsim_qiskit)
circ_fsim_qiskit

On the other hand, `cirq` does support the fsim gate, and a translation to `cirq` gates leaves the fsim gate unchanged.

In [21]:
circ_fsim_cirq = qf.circuit_translate(circ_fsim, targets=qf.CIRQ_GATES)
assert qf.circuits_close(circ_fsim, circ_fsim_cirq)
circ_fsim_cirq

Here's a list of all current translations, with source and target gates.

In [22]:
print("Translation                               Source Gate           Target Gates")
print()

for trans in qf.TRANSLATIONS:
  name = trans.__name__
  source = qf.translation_source_gate(trans).__name__
  targets = ', '.join(gate.__name__ for gate in qf.translation_target_gates(trans))
  print(f'{name:<40}  {source:<20}  {targets}')

Translation                               Source Gate           Target Gates

translate_IdentityGate_to_I               IdentityGate          I
translate_MultiSwapGate_to_swap_network   MultiSwapGate         Swap
translate_ReversalGate_to_swap_network    ReversalGate          Swap
translate_QFTGate                         QFTGate               H, CZPow, Swap
translate_InvQFTGate                      InvQFTGate            H, CZPow, Swap
translate_PauliGate                       PauliGate             CNot, XPow, YPow, ZPow
translate_DiagonalGate                    DiagonalGate          Rz, CNot
translate_MultiplexedRzGate               MultiplexedRzGate     Rz, CNot
translate_MultiplexedRyGate               MultiplexedRyGate     V, V_H, MultiplexedRzGate
translate_x_to_tx                         X                     XPow
translate_y_to_ty                         Y                     YPow
translate_z_to_tz                         Z                     ZPow
translate_s_to_tz             